# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

mkdir: data: File exists


In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  75569      0 --:--:-- --:--:-- --:--:--     0-- 75755


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   684k      0 --:--:-- --:--:-- --:--:--  691k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [1]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Property 'headlines' already exists in node 'a27438'. Skipping!
Property 'headlines' already exists in node 'd8fce0'. Skipping!


Applying HeadlineSplitter:   0%|          | 0/14 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

...

Applying CustomNodeFilter:   0%|          | 0/24 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/50 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'a27438'. Skipping!
Property 'summary_embedding' already exists in node 'd8fce0'. Skipping!
Property 'themes' already exists in node 'dc207f'. Skipping!
Property 'themes' already exists in node 'afcafd'. Skipping!
Property 'themes' already exists in node 'a770ef'. Skipping!
Property 'themes' already exists in node 'ada761'. Skipping!
Property 'themes' already exists in node '8d1627'. Skipping!
Property 'entities' already exists in node 'afcafd'. Skipping!
Property 'entities' already exists in node 'a770ef'. Skipping!
Property 'entities' already exists in node '8d1627'. Skipping!
Property 'themes' already exists in node 'd15156'. Skipping!
Property 'entities' already exists in node 'd15156'. Skipping!
Property 'entities' already exists in node 'dc207f'. Skipping!
Property 'themes' already exists in node '2ccce0'. Skipping!
Property 'themes' already exists in node 'c2df2d'. Skipping!
Property 'entities' already exists in node 'ada761'. 

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 26, relationships: 294)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 70)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [13]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

The three types of query synthesizers are:

1. SingleHopSpecificQuerySynthesizer: Generates straightforward, direct questions that can be answered using a single piece of information from the knowledge graph. For example: "What is AI?" or "When was GPT-4 released?"

2. MultiHopAbstractQuerySynthesizer: Creates more complex questions that require connecting multiple pieces of information and some level of abstraction or reasoning. For example: "What are the ethical concerns related to the gullibility of language models?"

3. MultiHopSpecificQuerySynthesizer: Generates detailed questions that need multiple specific pieces of information to be connected, but stays focused on concrete facts rather than abstract concepts. For example: "How has the development of Claude 3 and other multimodal models impacted AI in 2024?"




Finally, we can use our `TestSetGenerator` to generate our testset!

In [14]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,what microsoft research do with LLMs?,[Code may be the best application The ethics o...,Microsoft Research has produced better-than-GP...,single_hop_specifc_query_synthesizer
1,¿Por qué es más fácil para los LLMs manejar la...,[Based Development As a computer scientist and...,Los LLMs son más efectivos en la escritura de ...,single_hop_specifc_query_synthesizer
2,what happen with AI in 2023?,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,what midjourney do in 2023 and how it impact a...,[easy to follow. The rest of the document incl...,Midjourney 5.1 was one of the top entries in t...,single_hop_specifc_query_synthesizer
4,What advancements have been made in the Claude...,[Prompt driven app generation is a commodity a...,"Anthropic's Claude 3 series launched in March,...",single_hop_specifc_query_synthesizer
5,How have pricing changes in AI models influenc...,[<1-hop>\n\nPrompt driven app generation is a ...,Pricing changes in AI models have significantl...,multi_hop_abstract_query_synthesizer
6,How does test-driven development for system pr...,[<1-hop>\n\nPrompt driven app generation is a ...,Test-driven development for system prompts enh...,multi_hop_abstract_query_synthesizer
7,How have pricing changes in AI models impacted...,[<1-hop>\n\nPrompt driven app generation is a ...,The pricing changes in AI models have signific...,multi_hop_abstract_query_synthesizer
8,What advancements in language models surpassed...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, several advancements in language mode...",multi_hop_specific_query_synthesizer
9,How does the training cost and environmental i...,[<1-hop>\n\nday after that. DeepSeek v3 is a h...,"DeepSeek v3, a 685B parameter model, was train...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What Stability AI do in LLMs?,[Code may be the best application The ethics o...,Stability AI is one of the organizations that ...,single_hop_specifc_query_synthesizer
1,Wht are the advntages of using Python for LLMs?,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,What are some of the challenges associated wit...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"One of the challenges associated with GPT-4, a...",single_hop_specifc_query_synthesizer
3,What LLMs do?,[easy to follow. The rest of the document incl...,The document includes some of the clearest exp...,single_hop_specifc_query_synthesizer
4,How have inference-scaling reasoning models an...,[<1-hop>\n\nPrompt driven app generation is a ...,Inference-scaling reasoning models and multi-m...,multi_hop_abstract_query_synthesizer
5,How have inference-scaling reasoning models an...,[<1-hop>\n\nPrompt driven app generation is a ...,Inference-scaling reasoning models and multi-m...,multi_hop_abstract_query_synthesizer
6,How has the increased competition among AI mod...,[<1-hop>\n\nPrompt driven app generation is a ...,Increased competition among AI model providers...,multi_hop_abstract_query_synthesizer
7,How has the environmental impact of AI improve...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI has improved si...,multi_hop_abstract_query_synthesizer
8,How do the cost and efficiency improvements of...,[<1-hop>\n\ngets you OpenAI’s most expensive m...,GPT-4o is significantly more cost-effective th...,multi_hop_specific_query_synthesizer
9,How do the capabilities of GPT-4o and Claude A...,[<1-hop>\n\nreasoning patterns. Another common...,The capabilities of GPT-4o and Claude Artifact...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [32]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [33]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [34]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [37]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents refer to AI systems that are often envisioned to act on behalf of users. However, the term is vague and lacks a single, clear definition. There are two main categories of thought regarding agents: one sees them as systems that can perform tasks for users (like a travel agent), while the other thinks of them as LLMs (large language models) that utilize tools to solve problems. Despite the excitement surrounding AI agents, there have been few examples of them being successfully implemented in production. The term is complicated by issues of "gullibility" in AI systems, as they sometimes cannot distinguish truth from fiction.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

# Highlight what each evaluator is evaluating.

- `qa_evaluator`: Evaluates the quality of question answering by checking if the response correctly answers the given question using the provided reference answer
- `labeled_helpfulness_evaluator`: Evaluates how helpful the response is to the user by comparing the prediction against the reference answer and input question
- `dope_or_nope_evaluator`: Evaluates whether the response is "dope, lit, or cool" based on a subjective assessment of style and engagement


qa_evaluator and labeled_helpfulness_evaluator use the same context (qa one is preset thanx to LangChain). One is testing for truthfulness and other for usefulness

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'artistic-airplane-96' at:
https://smith.langchain.com/o/d4a9b1bb-13f2-4218-9bc4-42e74c3a8e71/datasets/e5414c92-2186-49ba-a0e0-93714e98ebf5/compare?selectedSessions=4077b184-5efe-4d94-b74a-022c64f15c61




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How did the advancements in GPT-4 capabilities...,The advancements in GPT-4 capabilities in 2024...,None,"In 2024, the advancements in GPT-4 capabilitie...",1,0,0,4.120632,d98cf2f7-7c37-45ab-b1df-e4b600ac87ea,2519d856-c805-45b3-98ed-b8e039d22c80
1,What were the key developments in Large Langua...,Key developments in Large Language Models (LLM...,None,"In 2023, Large Language Models (LLMs) experien...",1,0,1,3.422827,d757119b-830f-4d1c-9fa7-8a1e925ac114,d952c6f9-b8e2-40cc-a47a-44dd29e34ca7
2,How do the capabilities of GPT-4o and Claude A...,I don't know.,None,The capabilities of GPT-4o and Claude Artifact...,0,0,0,1.032603,af545a9d-7d3d-481e-8e51-1860e02464f2,7bd6300c-722c-4e0b-ab44-e1b7abb18258
3,How do the cost and efficiency improvements of...,The cost of OpenAI’s GPT-4o is $2.50 per milli...,None,GPT-4o is significantly more cost-effective th...,1,1,1,3.777268,c964cc18-8501-43ee-88ad-185ecfc4e338,2395e09e-2f0b-47a2-9cbd-2790de12605c
4,How has the environmental impact of AI improve...,The environmental impact of AI has improved du...,None,The environmental impact of AI has improved si...,1,0,0,4.198028,08b332d9-6bc6-4a50-9d9a-ebbc0e2c1f9f,6fcc7b3c-05dd-4489-8606-fc89f3f0b3a9
5,How has the increased competition among AI mod...,The increased competition among AI model provi...,None,Increased competition among AI model providers...,1,0,0,3.376928,0262dcdc-3509-4808-b431-c2fa44fe0577,39e765dc-457b-4331-824a-22e4efe84d10
6,How have inference-scaling reasoning models an...,Inference-scaling reasoning models have extend...,None,Inference-scaling reasoning models and multi-m...,1,0,1,6.771705,7520fc01-6303-4956-8e8b-64df3af09bd9,1b76d9b4-7f57-4bf5-8380-e8532042b0be
7,How have inference-scaling reasoning models an...,"Inference-scaling reasoning models, such as Op...",None,Inference-scaling reasoning models and multi-m...,1,0,1,4.443836,fe73edd2-8cca-48ed-9a5e-2be6ea47bcc8,37460430-180b-4d7b-b685-f41541aedf55
8,What LLMs do?,I don't know.,None,The document includes some of the clearest exp...,0,0,0,1.117360,9e4f3f08-d568-4685-83ee-fcfe67c6c2ec,7d24646e-a20b-4a47-94e2-4f82c26777cd
9,What are some of the challenges associated wit...,Some of the challenges associated with GPT-4 a...,None,"One of the challenges associated with GPT-4, a...",0,0,0,2.219792,95319df4-6e9b-4fa2-8040-8c5419777ec8,4f26f4a2-3e60-4230-b133-5b0258e3bad8


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [38]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [39]:
rag_documents = docs

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

Modifying chunk size affects performance in several key ways:

1. Larger chunks (1000 chars vs smaller) provide more context in each segment, allowing the model to better understand 
   the full context and relationships between concepts

2. With larger chunks but small overlap (50 chars), we reduce redundancy while still maintaining continuity between segments

3. This chunking approach helps balance between:
   - Having enough context for accurate answers
   - Keeping chunks small enough for efficient embedding and retrieval
   - Maintaining semantic coherence within each chunk

The 1000/50 split we chose aims to optimize this tradeoff for our RAG application's performance.



In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

Modifying the embedding model (text-embedding-3-large) affects performance in several ways:

1. Different embedding models have varying capabilities in capturing semantic meaning
   - More advanced models like text-embedding-3-large can better understand nuanced relationships
   - This impacts the quality of vector representations used for retrieval

2. The dimensionality and quality of embeddings affects:
   - How accurately similar documents can be retrieved
   - The precision of semantic search results
   - The overall relevance of retrieved context

3. There are performance tradeoffs:
   - More sophisticated embedding models may be slower but produce better results
   - Simpler models might be faster but less accurate
   - The choice depends on application needs (speed vs accuracy)

Using text-embedding-3-large provides high quality embeddings for accurate retrieval, though at potentially higher computational cost.



In [42]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [43]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [44]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [45]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Agents, man, are this super vague concept in the AI world, like trying to catch smoke with your bare hands! People toss the term around to describe AI systems that can act on your behalf—think of a travel agent or an LLM running tools to solve problems. But here's the kicker: the meaning of “agents” varies wildly, and it's still kind of in that perpetual “coming soon” phase. They’re supposed to do some cool stuff, but with the whole gullibility issue hanging over them, who knows when they’ll really shine? So, in a nutshell, agents are these elusive AI helpers that we’re still trying to pin down!"

Finally, we can evaluate the new chain on the same test set!

In [46]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'artistic-sail-67' at:
https://smith.langchain.com/o/d4a9b1bb-13f2-4218-9bc4-42e74c3a8e71/datasets/52a14930-c209-4faa-be02-3e90828c9d58/compare?selectedSessions=782c510c-610b-43b8-a7bf-8c44af205151




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Wht r the key advancemnts in LLMs in 2023 and ...,"Yo, in 2023, we saw some sick advancements in ...",None,"In 2023, Large Language Models (LLMs) saw sign...",1,0,1,2.836751,49e20499-93f5-4edb-a007-5cea5fef0296,716416cd-7857-4c4a-8b96-99c682357b24
1,What advancements in large language models in ...,"Yo, check it! In 2024, the scene got lit with ...",None,"In 2024, the GPT-4 barrier was comprehensively...",1,0,1,4.160744,b98a2a3b-378d-419e-8272-54bb954261b4,01677707-aeff-4158-9f8f-e1db6f05071c
2,How does GPT-4o differ from GPT-4 in terms of ...,"Alright, let's break it down! GPT-4o is like t...",None,GPT-4o differs from GPT-4 in that it can run w...,1,1,1,3.042767,817c688d-de34-43c6-832c-3df878c9b1ef,2a621f75-0052-4662-8e62-6edd265deed4
3,How does the concept of 'Vibes Based Developme...,"Yo, 'Vibes Based Development' is like the ulti...",None,The concept of 'Vibes Based Development' relat...,1,1,1,3.657645,e896bdea-55e8-495d-8590-49ebf01f9f22,d738561b-925e-4b30-87ef-3e316274d89f
4,How do the challenges of understanding and con...,"Yo, the challenges of understanding and contro...",None,The challenges of understanding and controllin...,1,0,1,8.745207,593063dc-44ba-4181-bc96-4a1ddb001197,e518c129-4dc4-4211-aa56-9f7e64989073
5,How have advancements in model training costs ...,"Yo, let me break it down for you! Over the pas...",None,Advancements in model training costs and envir...,1,0,1,4.290057,535696e8-6cd2-4b9d-9912-ba6b8e7e2138,d7f01b08-6a86-412a-9806-d85093f45f88
6,How do the challenges of understanding and con...,"Yo, the challenges of dealing with LLMs as 'bl...",None,The challenges of understanding and controllin...,0,0,1,4.223417,f8cd4329-98f9-40fd-b26f-4ccad0671241,a3739e9a-e279-4071-98f8-b462189fff3e
7,How do the challenges of understanding and con...,"Yo, the whole black box vibe of LLMs makes the...",None,The challenges of understanding and controllin...,1,0,1,8.393896,a7086479-97a8-43f4-a0bc-6f04dafaae02,0fd47756-1a06-4d9b-bad4-55780bfdf5be
8,Wht is the role of DALL-E 3 in the context of ...,I don't know.,None,The context mentions 'Now add a walrus: Prompt...,0,0,0,1.076571,2d4ddecd-8c39-414e-8bde-f7e9d554a809,4a6d4feb-0f9e-4c24-804f-520536459d6c
9,what happen with AI in 2023?,"Yo, 2023 was the breakthrough year for Large L...",None,2023 was the breakthrough year for Large Langu...,1,1,1,4.448919,5906dfa4-6f7a-41ff-8c74-3c0a899e1996,a9a8a692-b189-4ef7-8fe9-a12390846798


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

# ![State of AI Across Years](state-of-ai-across-years.png)

Much more Dope! Increase also in correctness and helpfulness but these are less pronounced. This is likely because:
1. The "dopeness" prompt specifically targets making responses more engaging and conversational, which directly impacts the DOPE metric
2. The improved engagement and clarity from the conversational style indirectly helps with correctness and helpfulness - possibly the increased "dopeness" means answers are longer making it easier for the content to include correct or helpful information?
3. However, since we didn't specifically optimize for correctness/helpfulness in our prompt changes, these metrics saw smaller gains
